# T-Teste para duas populações
**Objetivo: ao final desta aula, o aluno saberá aplicar e interpretar um t-teste para avaliar se as médias de duas populações são diferentes**


Até o momento, sabemos que nem sempre nossos dados condizem com nossos modelos. Para isso, temos feito o seguinte procedimento: primeiro, coletamos dados gerando uma amostra de tamanho $N$, e calculamos sua média amostral $\bar{X}$. Daí, se sabemos a média $\mu$ e a variância $\sigma^2$ teóricas do nosso modelo, então usamos essa informação para inferir que:

$$
\bar{X} \sim N(\mu, \sigma^2/N)
$$

Isso é equivalente a calcular a estatística $Z = (\bar{X}-\mu)/\sigma$ e saber que ela segue uma distribuição normal padrão, isto é, $Z \sim N(0,1)$.

Caso não saibamos a variância do modelo, passamos a usar a variância amostral $S=\sum_{n=1}^N (x_n-\bar{x})^2 / (N-1)$, e então, ao invés de calcular uma estatística $Z$, calculamos a estatística $T = (\bar{X}-\mu)/S$. A estatística $T$ segue uma distribuição t-student com $N-1$ graus de liberdade:

$$
T \sim T(N-1)
$$

Porém, em várias situações nós não temos um modelo pré-fixado e então geramos amostras. Em várias situações, temos duas amostras (diferentes e independentes) e queremos saber se elas foram geradas por modelos cuja média é a mesma. Veja a importância disso: podemos ter dois grupos de usuários no seu sistema - por exemplo, se temos duas estratégias de jogo, A e B, e queremos saber se elas são igualmente eficazes para pontuar.

Vamos fazer um estudo de caso para tornar esse exemplo mais claro. Após, vamos fazer nosso próprio teste com dados reais.

## O T-Teste para comparar duas populações: um estudo de caso

Esse estudo de caso está usando dados que foram cuidadosamente inventados para servirem como um bom exemplo. A cada etapa, refletiremos sobre o que foi feito.

Vamos começar com nossos dados. Temos 10 usuários usando a estratégia A. Para eles, $\bar{A}=100$ e $S_A=10$. Temos também 10 usuários usando a estratégia B, e sua pontuação tem média amostral $\bar{B}=105$ e desvio padrão amostral $S_B=30$. A questão que queremos resolver é: será que a média da pontuação com a estratégia A é diferente da média usando a estratégia B?


### Exercício 1: Avalie a hipótese nula

Começamos então com nossa hipótese nula. Avalie a hipótese nula abaixo. Você concorda com essa formulação para a hipótese nula? Caso necessário, reformule a hipótese.

$$
H_0: \mu_A = \mu_B
$$

### Exercício 2: A distribuição da diferença

Se nossa hipótese nula está correta, então temos uma nova variável aleatória aparecendo.

Essa variável é: $\bar{M}=\bar{A}-\bar{B}$.

Qual deve ser:

1. A média de $\bar{M}$?
2. O desvio padrão amostral de $\bar{M}$? (lembre-se da propriedade: $\text{Var}(x+y) = \text{Var}(x) + \text{Var}(y) + 2\text{Cov}(x,y)$)

### Exercício 3: calculando a estatística T

Agora, temos uma variável $\bar{M}$ para a qual sabemos a média teórica (*sob a hipótese de que H_0 está correta*) e o desvio padrão amostral.

Temos agora duas alternativas. Ou calculamos uma estatística $Z$ ou calculamos uma estatística $T$ para nossos dados. Qual dos dois deveríamos calcular, e por que?

Calcule a estatística!!!

### Exercício 4: calculando a probabilidade de encontrarmos nossas medições para $M$ dada a hipótese nula

Agora, podemos usar `stats.t.cdf` para estimar a probabilidade de encontrarmos um valor de $\bar{M}$ ao menos tão distante de 0 quanto o que foi encontrado. Se o número de amostras dos grupos A e B for igual, então usamos `df=NA+NB-2`.

O código abaixo foi feito para calcular essa probabilidade.

Acompanhe o código e responda às perguntas que estão nos comentários

In [7]:
import scipy.stats as stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# (a) O que cada uma dessas variáveis representa?
Abar = 100
As = 10
NA = 30
Bbar = 105
Bs = 30
NB = 30

# (b) Desvio padrão de Abar e Bbar. Por que estamos dividindo por N**0.5 e não por N?
Sigma_Abar = As/np.sqrt(NA)
Sigma_Bbar = Bs/np.sqrt(NB)
print(Sigma_Abar, Sigma_Bbar)

# (c) Média e desvio padrão de Mbar. Por que temos uma raiz quadrada em Sigma_M?
Mbar = Abar - Bbar
Sigma_M = np.sqrt( (Sigma_Abar**2) + (Sigma_Bbar**2) )

# (d) Estatística T. Por que não usamos Mbar-mu no numerador?
T = Mbar / Sigma_M

# (e) CDF da distribuição T. Por que estamos multiplicando o resultado por 2?
p = stats.t.cdf(T, df=NA+NB-2)*2
print(T,p)

1.8257418583505538 5.477225575051661
-0.8660254037844386 0.39004459129895575


### Exercício 5: usando o pacote Python

No `scipy.stats`, já temos uma função que realiza todos os procedimentos acima por nós. Analisando a chamada da função e seus resultados, o que significam as variáveis `t` e `p`?

In [4]:
t, p = stats.ttest_ind_from_stats(mean1=Abar, std1=As, mean2=Bbar, std2=Bs, nobs1=NA, nobs2=NB)
print(t, p)

-0.8660254037844386 0.39004459129895575


### Exercício 6: interpretando o valor `p`

De acordo com os resultados acima: 

(a) qual é o p-valor relacionado à hipótese nula?

(b) a hipótese nula deve ser rejeitada para $\alpha=5\%$?

# Um caso real

No semestre passado, a turma de BCC do Insper fez uma atividade de interfaces que consistia em mostrar um texto na tela e então contar quanto tempo o usuário levaria para responder a uma pergunta:

<img src="atividade_devlife.png">

Para esta atividade, vamos usar a técnica que aprendemos hoje para verificar se as duas interfaces fizeram diferença. Inicialmente, vamos estabelecer que queremos rejeitar a hipótese nula com $\alpha=5\%$, e, agora, vamos partir para a atividade!

## Passo 1: limpeza de dados

Os dados foram copiados e colados da tabela que foi gerada pelos próprios alunos na atividade do semestre passado. Isso significa que podem haver inconsistências e erros. Analise os dados e corrija as inconsistências, mas *sem eliminá-los*.

In [14]:
df = pd.read_csv('devlife_unidades_de_oxigenio.csv')
df.head()

,Resposta,Tempo,Variante
0,Correta,15.00,A
1,Correta,33.00,A
2,Correta,29.00,A
3,Correta,33.48,A
4,Correta,54.00,A


## Passo 2: a primeira hipótese nula

Nossa primeira pergunta é: *será que a média de tempo dos usuários da variante A foi diferente da média de tempo da variante B?*. Para respondê-la:

(a) Mostre a média de tempo dos usuários da variante A e da variante B em uma figura.

(b) A função `ttest_ind` recebe como parâmetros os valores de duas amostras independentes e calcula todos os parâmetros necessários (médias, variâncias, etc.) e retorna a estatística t e um p-valor relacionado. Na verdade, essa função literalmente calcula a média e desvio padrão amostrais e então chama a função `ttest_ind_from_stats` que usamos acima. Usando a função `ttest_ind`, determine qual é a sua hipótese nula relacionada à pergunta que queremos responder, e calcule um p-valor para ela.

(c) De acordo com o p-valor que você encontrou e o valor $\alpha$ que determinamos, qual é a resposta para a pergunta que fizemos?

In [15]:
A = df[ df['Variante']=='A' ]['Tempo']
B = df[ df['Variante']=='B' ]['Tempo']
print(np.mean(A), np.mean(B))

54.65533333333334 65.15229411764706


In [16]:
# Exemplo de uso da função ttest_ind:
t,p = stats.ttest_ind(A,B)
print(t,p)

-0.7795047355855033 0.4417888506906762


In [8]:
# Faça seu código aqui

## Passo 3: uma segunda pergunta

Agora vamos refazer nossa pergunta. Siga as mesmas etapas que fizemos anteriormente para responder à pergunta: *considerando apenas os usuários que acertaram à questão, há diferença na média de tempos quando usamos as interfaces A ou B*? 

In [18]:
# Faça seu código aqui
A = df[ df['Variante']=='A' ]
filtroA = A['Resposta']=='Correta'
A = A[filtroA]['Tempo']
B = df[ df['Variante']=='B' ]
filtroB = B['Resposta']=='Correta'
B = B[filtroB]['Tempo']
print(np.mean(A), np.mean(B))

52.77916666666667 80.96818181818182


In [20]:
# Exemplo de uso da função ttest_ind:
t,p = stats.ttest_ind(A,B)
print(t,p/2)

-1.730947817581554 0.04906477447387571
